In [ ]:
import os
import csv
from countries_languages import country_to_language

import pandas as pd
from difflib import get_close_matches
import numpy as np
import re
from transliterate import translit
from unidecode import unidecode
import Levenshtein
import requests
import bs4
from bs4 import BeautifulSoup 
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import urllib.parse
from datetime import datetime

from urllib.parse import urlsplit

import wikipediaapi

import wikipedia

from urllib.parse import urlparse

import difflib

from datetime import datetime

import calendar

from selenium import webdriver
from selenium.webdriver.chrome.options import Options

import AssemblyHelpers
#from AssemblyHelpers import find_money_info_from_name


import requests
from bs4 import BeautifulSoup

In [ ]:
import os
os.getcwd()

In [ ]:
def load_csv_dataset(file_path):
    return pd.read_csv(file_path)


cyrillic_guys = load_csv_dataset('CSVs of edge cases - for F/DONE WORKING WITH/cyrillic_dudes.csv')
lookup_guys = load_csv_dataset('CSVs of edge cases - for F/lookup_required_dudes.csv')

online_lookup_1_found = load_csv_dataset('CSVs of edge cases - for F/Online lookup required - 1 name found.csv')
online_lookup_0_found = load_csv_dataset('CSVs of edge cases - for F/Online lookup required - 0 names found.csv')
online_lookup_multi_found = load_csv_dataset('CSVs of edge cases - for F/Online lookup required - multiple names found.csv')

In [ ]:
combined_df_large = load_csv_dataset('large_dataset_1 (combined DF).csv')

large_dataset_said_0 = load_csv_dataset('CSVs of edge cases - for F/large_dataset_SAID_0.csv')

In [ ]:
#multi_names_used_large_dataset = combined_df_large[(combined_df_large['Status'] == 'SUCCESS') & (combined_df_large['Lookup Return Case'] == 'Used Large Dataset') & (combined_df_large['Name(s) Found'].astype(str).str.startswith('['))]#['Lookup Return Case'].value_counts()
multi_names_working = combined_df_large[(combined_df_large['Status'] == 'SUCCESS') & (combined_df_large['Lookup Return Case'] == 'working') & (combined_df_large['Name(s) Found'].astype(str).str.startswith('['))]#['Lookup Return Case'].value_counts()
multi_names_working.to_csv('multiple_names_working.csv')

In [ ]:
match_date_multi_guys = load_csv_dataset('CSVs of edge cases - for F/match_date_multi_guys.csv')
match_date_zero_guys = load_csv_dataset('CSVs of edge cases - for F/match_date_zero_guys.csv')

In [ ]:
#LOOKUP DATASETS

#leagues value
leagues_value = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/Most Updated Edited Transfermarkt Dataset.csv')
#leagues value large 
leagues_value_large = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/most_updated_transfermarkt_dataset.csv')

countries_codes = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/countries_and_codes.csv')

result_names_null_values = load_csv_dataset('/Users/finneganlaister-smith/Downloads/DEV ENVIRONMENT/data-science-jupyter-template-main/footy_project_market_value_rankings/create_dataset_1/nullnames_values.csv')
result_names_null_values = list(result_names_null_values['Name'])


# Helper methods

In [ ]:
#lookup required - 1 found - helper functions
from fuzzywuzzy import process

def find_close_matches(this_jersey, dataset_nationality, threshold=90):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]


def find_close_matches_variable(this_jersey, dataset_nationality, threshold):
    """
    Find close matches of `this_jersey` in `dataset_nationality` using Levenshtein distance.

    Args:
    - this_jersey (str): The string to find close matches for.
    - dataset_nationality (list): List of strings to search for close matches in.
    - threshold (int): Minimum similarity score required for a match (default is 90).

    Returns:
    - List of strings from `dataset_nationality` that are close matches to `this_jersey`.
    """
    close_matches = process.extract(this_jersey, dataset_nationality, limit=None)
    return [match[0] for match in close_matches if match[1] >= threshold]


def rearrange_name_asian(this_jersey):
    """
    Rearrange the name in `this_jersey` so that the last token comes first followed by a space.

    Args:
    - this_jersey (str): The name to rearrange.

    Returns:
    - Rearranged name string.
    """
    tokens = this_jersey.split()
    rearranged_name = tokens[-1] + " " + " ".join(tokens[:-1])
    return rearranged_name

import unicodedata


def normalize_name(name):
    """
    Normalize the given name by removing dashes, hyphens, apostrophes, and backticks,
    and replacing characters with accents with their unaccented counterparts.

    Args:
    - name (str): The name to normalize.

    Returns:
    - Normalized name string.
    """
    # Remove dashes, hyphens, apostrophes, and backticks
    name = re.sub(r'[-\'`]', '', name)

    # Replace characters with accents with their unaccented counterparts
    name = ''.join(char for char in unicodedata.normalize('NFD', name) if unicodedata.category(char) != 'Mn')

    return name

def switch_yi(string):
    if string.endswith('yi'):
        switched_string = string[:-2] + 'iy'
        return switched_string
    elif string.endswith('iy'):
        switched_string = string[:-2] + 'yi'
        return switched_string
    else:
        return string
    

def threshold_player_match(this_jersey, dataset_nationality):
    THRESHOLD_NUM = 89
    #stops when it returns a name.
    #if it doesnt find a match keep lowering the threshold until you find a match
    #but if you get to threshold of like 50 first you stop and just abandon ship    

    # Loop until someone is found or threshold goes below 50
    while THRESHOLD_NUM >= 50:
        matches = find_close_matches_variable(this_jersey, dataset_nationality, THRESHOLD_NUM)
        if matches:
            #print(f"Player is {this_jersey}. Found matches: {matches}. threshold is {THRESHOLD_NUM}")
            return matches, THRESHOLD_NUM
            #break
        else:
            THRESHOLD_NUM -= 1

    # If threshold reaches below 50 without finding any matches
    if THRESHOLD_NUM < 50:
        return [f"No matches found even with the lowest threshold."], this_jersey #jersey was {this_jersey}

def filter_matches_by_initial_and_name_start(this_jersey_tokens, matches):
    """
    Filter matches based on whether at least one token starts with the initial and at least one token starts with 
    the first character of the name, with additional conditions based on the tokens in this_jersey_tokens.

    Args:
    - this_jersey_tokens (list): List containing the initial and name parts of the jersey.
    - matches (list): List of matches to filter.

    Returns:
    - List of filtered matches.
    """
    filtered_matches = []
    
    # Check if this_jersey_tokens has multiple tokens
    if len(this_jersey_tokens) > 1:
        initial_starts_with = this_jersey_tokens[0]
        name_starts_with = this_jersey_tokens[1][0]

        # Additional logic for handling 'H' and 'Kh'
        if name_starts_with == 'H':
            if any(token.startswith('Kh') for token in this_jersey_tokens[1:]):
                name_starts_with = 'Kh'

        # Additional logic for handling multiple tokens
        for token in this_jersey_tokens[1:]:
            if token[0] != name_starts_with:
                initial_starts_with = None
                break
        
        for match in matches:
            match_tokens = match.split()
            initial_found = False
            name_start_found = False

            # Check if at least one token starts with the initial
            if initial_starts_with:
                for token in match_tokens:
                    if token.startswith(initial_starts_with):
                        initial_found = True
                        break

            # Check if at least one token starts with the first character of the name
            for token in match_tokens:
                if token.startswith(name_starts_with) or (name_starts_with == 'H' and token.startswith('Kh')):
                    name_start_found = True
                    break

            # Check if any token starts with any character of the hyphenated name
            if '-' in this_jersey_tokens[1]:
                hyphenated_parts = this_jersey_tokens[1].split('-')
                for part in hyphenated_parts:
                    for token in match_tokens:
                        if token.startswith(part[0]):
                            name_start_found = True
                            break

            # If both conditions are met, add the match to filtered_matches
            if initial_found and name_start_found:
                filtered_matches.append(match)

    else:  # Handle the case where this_jersey_tokens has only one token
        for match in matches:
            match_tokens = match.split()
            initial_found = False
            name_start_found = False

            # Check if at least one token starts with the initial
            for token in match_tokens:
                if token.startswith(this_jersey_tokens[0]):
                    initial_found = True
                    break

            # Check if at least one token starts with the first character of the name
            for token in match_tokens:
                if token.startswith(this_jersey_tokens[0][0]):
                    name_start_found = True
                    break

            # Check if any token starts with any character of the hyphenated name
            if '-' in this_jersey_tokens[0]:
                hyphenated_parts = this_jersey_tokens[0].split('-')
                for part in hyphenated_parts:
                    for token in match_tokens:
                        if token.startswith(part[0]):
                            name_start_found = True
                            break

            # If both conditions are met, add the match to filtered_matches
            if initial_found and name_start_found:
                filtered_matches.append(match)

    return filtered_matches


def check_last_token_match(this_jersey, matches):
    """
    Check if the last token of each match matches the jersey with variations like accents or substitutions.

    Args:
    - this_jersey (str): The original name being searched for.
    - matches (list): Set of names matching the jersey.

    Returns:
    - List of matches where the last token matches the jersey with variations.
    """
    def remove_accents(input_str):
        """
        Remove accents from characters in a string.
        """
        nfkd_form = unicodedata.normalize('NFKD', input_str)
        return ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    def apply_substitutions(input_str):
        """
        Apply specific character substitutions in a string.
        """
        substitutions = {'g': 'h', 'ij': 'y', 'zh': 'j', 'ae': 'aye', 'H': 'Kh'}
        for key, value in substitutions.items():
            input_str = input_str.replace(key, value)
        return input_str

    jersey_last_token = this_jersey.split()[-1]

    matching_matches = []
    for match in matches:
        match_last_token = match.split()[-1]

        # Check if the last token matches the jersey (ignoring accents)
        if remove_accents(match_last_token) == remove_accents(jersey_last_token):
            matching_matches.append(match)
        else:
            # Check if the last token matches the jersey with specific character substitutions
            if apply_substitutions(match_last_token) == apply_substitutions(jersey_last_token):
                matching_matches.append(match)

    return matching_matches

def filter_multi_word_matches_by_jersey_tokens(jersey_tokens, potential_matches):
    filtered_matches = []

    for match in potential_matches:
        # Split each match into tokens
        match_tokens = match.split()

        # Count the number of tokens that start with each character in jersey_tokens
        match_token_start_chars = {token[0] for token in match_tokens}

        # Check if the match contains at least one token for each character in jersey_tokens
        if all(token[0] in match_token_start_chars for token in jersey_tokens):
            filtered_matches.append(match)

    return filtered_matches




# Lookup Required

In [ ]:
lookup_guys

# Online Lookup Required

In [ ]:
from Levenshtein import distance

#Helper methods
def rearrange_name(name):
    tokens = name.split()
    if len(tokens) > 1:
        last_token = tokens.pop()
        tokens.insert(0, last_token)
        return ' '.join(tokens)
    else:
        return name
    

def closest_string(this_jersey, dataset_nationality):
    closest_distance = float('inf')
    closest_strings = []
    
    # Splitting this_jersey into tokens
    tokens = this_jersey.split(' ')
    
    # If there's only one token, return all names where the single token is in that name
    if len(tokens) == 1:
        for nationality in dataset_nationality:
            if tokens[0] in nationality:
                closest_strings.append(nationality)
            elif tokens[0] in unidecode(nationality):
                closest_strings.append(nationality)
        return closest_strings

    # Calculate distances for all names
    for nationality in dataset_nationality:
        distance = Levenshtein.distance(this_jersey, nationality)
        if distance < closest_distance:
            closest_distance = distance
            closest_strings = [nationality]
        elif distance == closest_distance:
            closest_strings.append(nationality)

    # If there are multiple names with the same distance,
    # and some of them have the same last name, include all of them
    if len(closest_strings) > 1:
        last_names = [name.split()[-1] for name in closest_strings]
        unique_last_names = set(last_names)
        if len(unique_last_names) == 1:
            return closest_strings
    
    return closest_strings

def remove_vowels(text):
    vowels = 'aeoAEIOU'
    return ''.join(char for char in text if char not in vowels)


def filter_matches(correct_matches, closest):
    filtered_matches = []
    for match in correct_matches:
        if match.lower() == closest.lower():
            filtered_matches.append(match)
    return filtered_matches


def check_tokens(this_jersey, match):
    jersey_tokens = this_jersey.lower().split()
    match_tokens = match.lower().split()
    
    if all(token in match_tokens for token in jersey_tokens):
        return True
    elif all(token in jersey_tokens for token in match_tokens) and len(jersey_tokens) == len(match_tokens) + 1:
        return True
    elif all(token in match_tokens for token in jersey_tokens) and len(jersey_tokens) > len(match_tokens):
        return True
    else:
        return False
    
def count_diff_vowels(token1, token2):
    vowels = 'aeiou'
    count = 0
    for char1, char2 in zip(token1, token2):
        if char1.lower() in vowels and char2.lower() in vowels:
            if char1.lower() != char2.lower():
                count += 1
    return count

def filter_names_by_tokens(names, tokens):
    filtered_names = []
    for name in names:
        if all(token in name for token in tokens):
            filtered_names.append(name)
    return filtered_names

In [ ]:
dataframe_matches_2 = pd.DataFrame()
#for index, row in online_lookup_1_found.iterrows():
for i in range(1200, 1258):
    status_row = 'Fail'
    matches_final = []
    correct_matches = ''
    filtered_matches = ''
    print(f"**{i}**")
    index = i
    row = online_lookup_1_found.iloc[index]
    this_row_success_tally = 0
    this_row_multi_tally = 0
    this_row_fail_tally = 0

    name_list_matches = []

    this_jersey = row['ORIGINAL JERSEY']

    if this_jersey == 'R Myratberdiýew':
        this_jersey = 'R Muratberdyev'
    elif this_jersey == 'E ilitao':
        this_jersey = 'Éder Militão'
    elif this_jersey == 'Feliciano Jone':
        this_jersey = "Nené"
    elif this_jersey == 'E Håland':
        this_jersey = 'E Haaland'
    elif this_jersey == 'Phm Thành Lng':
        this_jersey = 'Pham Thành Luong'
    elif this_jersey == 'Þ Helgason':
        this_jersey = 'T Helgason'
    
        
    
    nationality = row['Nationality']
    nationality_code = row['Team Country Code']
    if nationality_code == 'BVI':
        nationality_code = ' BVI'
    country_name = countries_codes[countries_codes[' Code'] == nationality_code]['Country'].unique()[0]
    if nationality_code == ' DR':
        nationality_code = 'DR'
    season = row['Season']
    match_date = row['Date']
    dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == nationality_code]['Name'].unique()

    if this_jersey.endswith('yi'):
        this_jersey = switch_yi(this_jersey)
    elif this_jersey.endswith('iy'):
        this_jersey = switch_yi(this_jersey)

    name_type = ''
    # if (nationality_code == 'KR') | (nationality_code == 'CN') | (nationality_code == 'TW') | (nationality_code == 'SG') | (nationality_code == 'KP'):
    #     this_jersey = rearrange_name_asian(this_jersey)
    matches, threshold_num = threshold_player_match(this_jersey, dataset_nationality)

    this_jersey_tokens = this_jersey.split(' ')
    case_row = ''

    if len(this_jersey_tokens) >= 2:
        if len(this_jersey_tokens[0]) <= 2: #initial is at the start 
            case_row = 'Initial Plus Name'
            correct_matches = []
            for name in dataset_nationality:
                if (name.startswith(this_jersey_tokens[0])) & (name.endswith(this_jersey_tokens[-1])):
                    correct_matches.append(name)
                elif (name.startswith(this_jersey_tokens[0])) & (name.endswith(unidecode(this_jersey_tokens[-1]))):
                    correct_matches.append(name)
                elif (this_jersey_tokens[-1] in name) and any(token.startswith(this_jersey_tokens[0]) for token in name.split(' ')):
                    correct_matches.append(name)
                elif (this_jersey_tokens[-1] in name) and any(token.startswith(this_jersey_tokens[0].lower()) for token in name.split(' ')):
                    correct_matches.append(name)
                elif (this_jersey_tokens[-1] in unidecode(name)) and any(token.startswith(this_jersey_tokens[0]) for token in name.split(' ')):
                    correct_matches.append(name)
                
                #the full word is 
                elif name.startswith(this_jersey_tokens[0]):
                    name_tokens = name.split(' ')
                    if len(name_tokens) > 1:
                        last_token = name_tokens[-1]
                        target_token = this_jersey_tokens[-1]
                        if abs(len(last_token) - len(target_token)) <= 2 and distance(last_token, target_token) <= 2:
                            correct_matches.append(name)

            if len(correct_matches) >= 2:
                0==0 #row 1100, 1101
            elif len(correct_matches) == 1:
                0==0 #row 0
            else:
                0==0 #row 450
            #make sure the names remaining all start with an initial and ends with the jersey's name. 
            #be mindful of accents - if names match once you remove accents from the matches that's ok
        elif len(this_jersey_tokens[-1]) <= 2: #initial is at the end 
            case_row = 'Initial Plus Name'
            correct_matches = []
            for name in dataset_nationality:
                if (name.startswith(this_jersey_tokens[-1])) & (name.endswith(this_jersey_tokens[0])):
                    correct_matches.append(name)
                elif (name.startswith(this_jersey_tokens[-1])) & (name.endswith(unidecode(this_jersey_tokens[0]))):
                    correct_matches.append(name)
                

            if len(correct_matches) >= 2:
                0==0
            elif len(correct_matches) == 1:
                0==0
            else:
                0==0
        else: #multiple words
            case_row = 'Multiple Words'
            correct_matches = []
            for match in matches:
                match_tokens = re.split(r'[\s-]', match)
                all_tokens_valid = all(token in this_jersey_tokens or unidecode(token) in this_jersey_tokens for token in match_tokens)
                if all_tokens_valid:
                    #print(f'{match}. All tokens present.')
                    correct_matches.append(match)
                else:
                    extra_tokens = [token for token in match_tokens if token not in this_jersey_tokens and unidecode(token) not in this_jersey_tokens]
                    if not extra_tokens:
                        #print(f'{match}. All tokens present.')
                        correct_matches.append(match)
                    else:
                        #print(f'{match}. Extra tokens: {extra_tokens}')
                        correct_matches.append(match)
    else: #single word
        case_row = 'Single Word'
        correct_matches = []
        for match in matches:
            match_tokens = match.split(' ')
            last_token = match_tokens[-1]
            if match == this_jersey:
                correct_matches.append(match)
            if match.endswith(this_jersey):
                correct_matches.append(match)
            elif unidecode(match).endswith(this_jersey):
                correct_matches.append(match)
            elif unidecode(match).startswith(this_jersey):
                correct_matches.append(match)
            elif match.startswith(this_jersey):
                correct_matches.append(match)
            elif len(last_token) == len(this_jersey) and sum(1 for a, b in zip(last_token, this_jersey) if a != b and a.lower() == 'z' and b.lower() == 's') == 1:
                correct_matches.append(match) 
            elif len(last_token) == len(this_jersey) and sum(1 for a, b in zip(last_token, this_jersey) if a != b and a.lower() == 's' and b.lower() == 'z') == 1:
                correct_matches.append(match)

        if correct_matches == []:
            for match in matches:
                if this_jersey in match:
                    correct_matches.append(match)
            

    if correct_matches == []: #Vietnam case
        if nationality == 'Vietnam':
            matches_copy = matches[:]
            for match in matches_copy:
                matches.remove(match)  # Remove the original match from matches
                rearranged_match = rearrange_name_asian(match)  # Rearrange the name
                #print(rearranged_match)
                matches.append(rearranged_match)  # Add the rearranged name back to matches
                if remove_vowels(rearranged_match) == this_jersey:
                    
                    correct_matches.append(match)
    else:
        if nationality == 'Vietnam':
            matches_copy = matches[:]
            for match in matches_copy:
                
                matches.remove(match)  # Remove the original match from matches
                rearranged_match = rearrange_name_asian(match)  # Rearrange the name
                #print(rearranged_match)
                matches.append(rearranged_match)  # Add the rearranged name back to matches
                if remove_vowels(rearranged_match) == this_jersey:
                    
                    correct_matches.append(match)

    #print(f'player is {this_jersey}. after 1st round matches are {correct_matches}')
    if len(correct_matches) >= 2: 
        #print('multiple matches', this_jersey, correct_matches)
        if len(closest_string(this_jersey, dataset_nationality)) == 1: #was type == str
            if closest_string(this_jersey, dataset_nationality) in correct_matches:
                filtered_matches = filter_matches(correct_matches, closest_string(this_jersey, dataset_nationality))
                second_list = [match for match in correct_matches if check_tokens(this_jersey, match)]
                filtered_matches = list(set(filtered_matches + second_list))
                if filtered_matches == []:
                    print("x no good matches:", filtered_matches)  
                else:
                    #print("✔ Filtered matches:", filtered_matches) #Success
                    status_row = 'Success'
                    matches_final = filtered_matches
            else:
                filtered_matches = [match for match in correct_matches if check_tokens(this_jersey, match)]
                if filtered_matches == []:
                    for name in correct_matches:
                        if (this_jersey_tokens[-1] in unidecode(name)) and any(token.startswith(this_jersey_tokens[0]) for token in name.split(' ')):
                            filtered_matches.append(name)
                        elif (unidecode(this_jersey_tokens[-1]) in unidecode(name)) and any(token.startswith(this_jersey_tokens[0]) for token in name.split(' ')):
                            filtered_matches.append(name)
                    if filtered_matches == []:
                        for match in correct_matches:
                            match_tokens = re.split(r'[\s-]', match)
                            if match_tokens[-1].lower() == this_jersey_tokens[-1].lower() and match_tokens[0][0].lower() == this_jersey_tokens[0][0].lower():
                                filtered_matches.append(match)
                        
                        if filtered_matches == []:
                            0==0
                            if len(this_jersey_tokens) >= 2:
                                
                                token_counts = {token[0]: 0 for token in this_jersey_tokens}
        
                                # Iterate through each match in correct_matches
                                for match in correct_matches:
                                    # Create a copy of token_counts for each match
                                    match_token_counts = token_counts.copy()
                                    
                                    # Iterate through each token in the match
                                    for token in match:
                                        # Check if the token's first letter is in match_token_counts
                                        if token[0] in match_token_counts:
                                            # Increment the count for that letter
                                            match_token_counts[token[0]] += 1
                                    
                                    # Check if all letters in this match have at least one token
                                    if all(count > 0 for count in match_token_counts.values()):
                                        # Do something if all letters in this match have at least one token
                                        #print(match, "All letters in this match have at least one token.")
                                        filtered_matches.append(match)
                                        if match == 'Ngoc Hai Que':
                                            status_row = 'Success'
                                            matches_final = [match]

                                if filtered_matches == []:
                                    0==0
                                    #print("x No good matches", filtered_matches)
                        else:
                            #print("✔ Matches to keep:", filtered_matches) #Success
                            status_row = 'Success'
                            matches_final = filtered_matches
                    else:
                        #print("✔ Matches to keep:", filtered_matches) #Success
                        status_row = 'Success'
                        matches_final = filtered_matches
                else:
                    #print("✔ Matches to keep:", filtered_matches) #Success
                    status_row = 'Success'
                    matches_final = filtered_matches
        else: #returned multiple people 
            last_name = this_jersey.split()[-1]
            matching_names = [name for name in dataset_nationality if name.split()[-1] == last_name]
            if matching_names:
                matches_final = closest_string(this_jersey.split(' ')[-1], dataset_nationality)
            else:
                matches_final = closest_string(this_jersey, dataset_nationality)
            if this_jersey == 'd Leon':
                matches_final = correct_matches
            status_row = 'Success'
    elif len(correct_matches) == 1:
        #print('✔', this_jersey, correct_matches) #Success
        matches_final = correct_matches
        status_row = 'Success'
    elif len(correct_matches) == 0:
        0==0
        #print('no matches', this_jersey, matches)
    if correct_matches == []:
        if case_row == 'Single Word':
            0==0
        elif case_row == 'Multiple Words':
            print('here')
        elif case_row == 'Initial Plus Name':
            0==0
        else:
            print('where')

    if status_row == 'Success':
        if len(matches_final) == 1:
            if case_row == 'Multiple Words':
                candidate = matches_final[0]
                tokens_final_match = re.split(r'[\s-]', candidate)
                matching_tokens_count = sum(1 for token in tokens_final_match if token in this_jersey_tokens)
                if matching_tokens_count / len(tokens_final_match) < 0.5:
                    status_row = 'Lookup Still Required'
                    #print('X - reversed success', this_jersey, matches_final)
                    matches_final = []
                    this_jersey_tokens_new = re.split(r'[\s-]', this_jersey)
                    matching_tokens_count_new = sum(1 for token in tokens_final_match if token in this_jersey_tokens_new)
                    if matching_tokens_count_new / len(tokens_final_match) < 0.5:
                        0==0
                    else:
                        status_row = 'Success'
                        matches_final = [candidate]
                if len(tokens_final_match) == 2:
                    if matching_tokens_count == 1:
                        status_row = 'Lookup Still Required'
                        matches_final = [this_jersey, candidate]
        else:
            if len(this_jersey_tokens) > 1:
                if case_row == 'Initial Plus Name':
                    # Iterate over each item in matches_final
                    
                    for item in matches_final:
                        # Split the item into words
                        words = item.split()
                        # Check if at least one word, except the last one, starts with this_jersey_tokens[0]
                        if any(word.startswith(this_jersey_tokens[0]) for word in words[:-1]):
                            # If at least one word, except the last one, starts with this_jersey_tokens[0], do nothing
                            pass
                        else:
                            # If no word, except the last one, starts with this_jersey_tokens[0], remove the item from matches_final
                            matches_final.remove(item)
                                                                                                            # for match in matches_final:
                                                                                                            #     last_token_word = match.split(' ')[-1]
                                                                                                            #     print(last_token_word)
                                                                                                            #     if last_token_word == this_jersey_tokens[-1]:
                                                                                                            #         print('der')
                                                                                                            #get the last token of the word 
                                                                                                            #if the last token not the same as the last token 

    if status_row == 'Lookup Still Required':
        if len(correct_matches) == 1:
            if correct_matches[0] == row['Name(s) Found']:
                matches_final = correct_matches[0]
                # Extracting last tokens
                this_jersey_last_token = this_jersey.split()[-1]
                correct_matches_last_token = correct_matches[0].split()[-1]
                row_name_found_last_token = row['Name(s) Found'].split()[-1]
                # Check if both correct_matches_last_token and row_name_found_last_token are different from this_jersey_last_token
                if correct_matches_last_token != this_jersey_last_token and row_name_found_last_token != this_jersey_last_token:
                    matches_final = [this_jersey, correct_matches[0]]                
            elif row['Name(s) Found'] == closest_string(this_jersey, dataset_nationality)[0]:
                matches_final = row['Name(s) Found']
            
            # Extracting last tokens
            this_jersey_tokens = this_jersey.split()[-1]
            if len(matches_final) == 1:
                matches_final_tokens = matches_final[0].split()[-1]
            correct_matches_tokens = correct_matches[0].split()[-1]

            # Check conditions
            if this_jersey_tokens != matches_final_tokens and ((this_jersey_tokens == correct_matches_tokens) or 
            (count_diff_vowels(this_jersey_tokens, correct_matches_tokens) == 1)):
                #check if the last token of this_jersey is different from the last token of matches_final, AND its the same as the last token of correct matches
                matches_final = [correct_matches[0], filtered_matches , row['Name(s) Found'], this_jersey]


    if len(matches_final) > len(correct_matches) and \
    not any(item == this_jersey for item in matches_final) and \
    not any(item == this_jersey for item in correct_matches):
        filtered_matches_final = filter_names_by_tokens(matches_final, this_jersey)
        filtered_correct_matches = filter_names_by_tokens(correct_matches, this_jersey)
        #print("matches_final is longer than correct_matches and neither list has any items that are exact matches with this_jersey")
        #print(filtered_correct_matches, filtered_matches_final)
        matches_final = list(set(filtered_correct_matches + filtered_matches_final))
        status_row = 'Lookup Still Required'

    if status_row != 'Success':
        for index, string in enumerate(matches_final):
            if string == this_jersey:
                status_row = 'Success'
                matches_final = [string]  # Set matches_final to only contain the matching string
                break

    if status_row == 'Fail':
        if this_jersey == row['Name(s) Found']:
            matches_final = row['Name(s) Found']
            status_row = 'Success'

    print(f"jersey is {this_jersey}. STATUS = {status_row}.")
    if status_row == 'Lookup Still Required':
        print(f"FIRST ROUND WAS {correct_matches}, 2nd was {filtered_matches}. FINAL ROUND WAS {matches_final}. Row said {row['Name(s) Found']}")
    elif status_row == 'Success':
        print('FINAL MATCHES WERE', matches_final)
    else:
        print(f"FIRST ROUND WAS {correct_matches}, 2nd was {filtered_matches}. FINAL ROUND WAS {matches_final}. Row said {row['Name(s) Found']}")

#1200 -end
        #1210 - in case where it fails. if the name in the row equals the name on the jersey go with that name 



#ONCE READY

#change sim sang min row name to sim sang min cause thats correct 

    
    #IF ITS FAIL LOOK UP THE JERSEY (ONLINE)
    #IF ITS LOOKUP STILL REQUIRED LOOK UP THE FINAL MATCHES (ONLINE), and the thing in the row['Name(s) Found']
    #IF ITS SUCCESS 
            #IF MORE THAN 1 NAME FILTER USING AMTCH DATE
                #if that leaves nobody u have to look up the OG jersey
        #ONCE U HAVE 1 NAME FIND IN DB 

#provided you have a name 
    #look up that name + that season 
    #if there are no rows, you have to look them up online 
    


In [ ]:
#done cases 

#work on row 35
    #case is initial + name 
    #the match did start with Initial, and it ended with name but the real name has an extra U that the jersey was missing 
                
    #so if it starts with the initial and ends with a name 1 character differnt from the last name, that's ok too. 

#row 53 
    #if the name is only off between an S and a Z it should still be good 

#row 60 - single name 
    #if the MATCH was the closest name in the list. or maybe if the MATCH was the only word in dataset_nationality that contained the jersey_name

#row 62 - single name
    #REMEMBER TO ACCOUNT FOR ACCENTS 

#row 97 - if the jersey is S Cruz keep any matches ending in Santa Cruz

#222 - s and z thing should work 


#D Santos - 245
    #allow the token in match to start with the lower case version of the initial token   

#286 - d Leon
   #286 - the d Leon thing. i no longer get all the de Leon people  - fixed w ad hoc work 

#293 - Mohamed Eisa
    #if the last name is the same
    #and the first name starts with the same 2 letters
    #that should count 

#296 was no longer working but should be. the Mo kisa one 

#305   
    #this one should not work 
            
#309
    #SAME AS 305. SHOULD WORK BUT THEN TELL YOU IT'S NOT A GREAT MATCH AND TO LOOK IT UP AFTER 

#340 / 326
    #should return both people

    #374 - filter these names only one should remain       


#389
    #should return both of those people.


#474 and 470. IF THE NAME IN THE NAME(S) FOUND COLUMN IS THE SAME AS CORRECT MATCHES OR FILTERED MATCHES U GOTTA GO WITH THAT NAME 

    #478 save the names from close matches and filtered matches when it fails so you can try them later.      

    #589 should return names to search

#500-600
    #598 and #599 should return names to search
        #if the name is 2 names and you can only match 1 token 
            #and the non-matching tokens arent the same name
            #you should look up both names then. the jersey name and the one it found 

#611 should not work.

        #677 should only retunr 1 guy

#759 should not work.

        #752/738 thats the wrong person 

        #851 one of the matches is actually the right player 

        #879 feels match-able  - ad hoc fix

        #890 should be fixed by #374




#####LOGIC######
            
                                                            #FIRST FILTER - DONE
                                                                #if the name is initial + name - DONE
                                                                    
                                                                #if its multiple words in the name - DONE
                                                                    
                                                                #IF JUST A NAME - DONE

                                                                #multi-word names. that viet case - DONE

#SECOND FILTER 
    #check if the name it found is a better match than waht's in the name column 
        #if they're the same or if it is a better match move ahead with the new name 

        #if the row is a better match, but theyre both shite, what do you do? 
#correct_matches#match_tokens#this_jersey_tokens




In [ ]:
online_lookup_multi_found

In [ ]:
#online lookup required - multiple names found 


#if multiple names were found

    #if the Jersey has an initial 
        #keep all matches that start with the initial 

        #except if the initial is a lowercase d. then, make sure there is a short word (3 characters or less) starting with d followed by the Name 
        #so if its d Silva make sure theres a word startign with d then Silva soemwhere in the name. 

    #sadiqov case - what if the names are the same???
        #in this case you should just look up each name and use like whichever has the record from the saeason of the game 

    #do the match date lookup thing with each of the matches to check who played in that game. 
        #if you still have more than 1 name

            #look at the success dataset to find the record of each remaining guy
                #if its ivanovic and you find both B and D Ivanovic played according to transfermarkt
                
                #check if there's a record of those guys in that match / date (in our dataset)
                    
                    #if all but one guy is in the success dataset, the guy who is not there is the name you need to use.

                    #if none remain after checking the dataset just remove this row. but print that out. that would be weird

                    #if multiple remain after checking the success dataset(there were 3 names remaining, only found 1 record)
                        #print this out this is a special case.

        #if you are just left with one name using the match date lookup 

            #that's a success - find that guy in the database

In [ ]:
online_lookup_0_found


#look the guy up with no nationality in the offline DB

#look him up online with the season in question 

# CYRILLIC GUYS

In [ ]:
#helper methods

def query_google_1(input_string):
    # Replace spaces with plus signs to format the query string
    query = '+'.join(input_string.split())

    # URL for Google search
    url = f"https://www.google.com/search?q={query}"

    # Send GET request to Google
    response = requests.get(url)

    # Check if request was successful
    if response.status_code == 200:
        # Parse the HTML content
        soup = BeautifulSoup(response.text, 'html.parser')
        # Return the parsed HTML content
        return soup
    else:
        # If request fails, print error message
        print("Failed to fetch results from Google.")
        return None



def find_name_in_language_translate(name_1, language):
    status = 'Fail'
    name = ""
    input_string = f"translate Russian to {language}:{name_1}"
    try:
        resulting_html = query_google_1(input_string)
    except Exception as e:
        print("Couldn't get the right HTML:", e)
        return name, status
    
    try:
        body_jsmodel = resulting_html.find('body', attrs={'jsmodel': 'hspDDf '})
        potential_div = body_jsmodel.find_all('div')[7]
        #set_of_divs = potential_div.find_all('div') 
    except Exception as e:
        print("Could not find the correct divs:", e)
        return name, status

    try:
        # Find the div with the specified class name
        div_with_class = potential_div.find('div', class_='Va3FIb')
        # Or if you want to find all divs with the specified class name
        divs_with_class = potential_div.find_all('div', class_='Va3FIb')

        # Check if the div is found
        if div_with_class:
            status = 'Success'
            #print(div_with_class)
            name = (divs_with_class[0].text)
        else:
            print("Div with class 'Va3FIb' not found.")
            name = ""
    except Exception as e:
        print("Could not find the correct divs:", e)
    
    return name, status

def find_transfermarkt_links(html_content):
    # Regular expression patterns for transfermarkt URLs
    transfermarkt_patterns = [
        r"transfermarkt\.de",
        r"transfermarkt\.us",
        r"transfermarkt\.world"
    ]
    transfermarkt_links = []

    # Find all 'a' tags in the HTML content
    links = html_content.find_all('a', href=True)

    # Filter links that match transfermarkt patterns
    for link in links:
        for pattern in transfermarkt_patterns:
            if re.search(pattern, link['href']):
                transfermarkt_links.append(link['href'])
                break

    return transfermarkt_links


def extract_name_from_link(link):
        
    name_in_link = link.split('/')[3]
    name_in_link_tokens = name_in_link.split('-')
    capitalized_tokens = []

    for token in name_in_link_tokens:
        capitalized_tokens.append(token.capitalize())

    combined_string = ' '.join(capitalized_tokens)

    return combined_string


# def add_spaces_to_middle_caps(string):
#     # Split the string into tokens
#     tokens = string.split()
    
#     # Check if there's only one token
#     if len(tokens) == 1:
#         token = tokens[0]
        
#         # Check if there are capital letters in the middle of the token
#         if any(c.isupper() for c in token[1:-1]):
#             # Add spaces before each capital letter in the middle
#             modified_token = ''
#             for char in token:
#                 if char.isupper() and token.index(char) != 0 and token.index(char) != len(token) - 1:
#                     modified_token += ' ' + char
#                 else:
#                     modified_token += char
#             return modified_token
#     return string
def add_spaces_to_middle_caps(string):
    # Split the string into tokens
    tokens = string.split()
    
    # Check if there's only one token
    if len(tokens) == 1:
        token = tokens[0]
        
        # Check if there are capital letters in the middle of the token
        if any(c.isupper() for c in token[1:-1]):
            # Add spaces before each capital letter in the middle
            modified_token = ''
            for index, char in enumerate(token):
                if char.isupper() and index != 0 and index != len(token) - 1:
                    modified_token += ' ' + char
                else:
                    modified_token += char
            return modified_token
    return string

from translate import Translator

def translate_cyrillic_to_english(text):
    translator = Translator(to_lang="en", from_lang="ru")
    translation = translator.translate(text)
    return translation



def process_google_query(input_text):
    html_result = query_google_1(input_text)
    if html_result:
        transfermarkt_links = find_transfermarkt_links(html_result)
        if transfermarkt_links:
            names_links = []
            for link in transfermarkt_links:
                if link.startswith('/url?q='):
                    link = link.split('/url?q=')[1]
                name_from_link = extract_name_from_link(link)
                names_links.append(name_from_link)
            if len(names_links) == 1:
                name = names_links[0]
                return name
            elif len(names_links) >= 2:
                name_list_matches = list(set(names_links))
                return name_list_matches
            else:
                return 0  # Nobody found
        else:
            return 0  # No links found
    else:
        return 0  # No HTML result

def vet_tokens_names(name, player):
    player_original = player.copy()
    original_name_tokens = name.split(' ')
    print(original_name_tokens)
    for player_individual in player:
        #print(player)
        this_player_tokens = player_individual.split(' ')
        
        for token in original_name_tokens:
            if token == '':
                0==0
            else:
                first_letter = token[0]
                #print(initial_token)
                token_found = False
                for token_player in this_player_tokens:
                    if token_player.startswith(first_letter):
                        
                        if levenshtein_distance(token_player, token) >= len(token_player):
                            #print(f'did not match {token_player} with {token}')
                            0==0
                        else:
                            #print(f'matched {token_player} with {token}')
                            #print(token_player)
                            token_found = True
                            break
                if not token_found:
                    #print(player)
                    if player_individual in player:
                        player.remove(player_individual)
    #print('banana', player)
    if player  == []:
        player = player_original
        #print('razz', player)
        for player_individual in player:
            this_player_tokens = re.split(r'\s+|-|–', player_individual)
            #print('apple', this_player_tokens)
            for token in original_name_tokens:
                if token == '':
                    0==0
                else:
                    first_letter = token[0]
                    token_found = False
                    for token_player in this_player_tokens:
                        if token_player.startswith(first_letter):
                            #print('pine', token_player)
                            if levenshtein_distance(token_player, token) >= len(token_player):
                                #print(f'did not match {token_player} with {token}')
                                0==0
                            else:
                                #print(f'matched {token_player} with {token}')
                                #print(token_player)
                                token_found = True
                                break
                if not token_found:
                    player.remove(player_individual)
                    break

    print(player)
    return player

def remove_duplicates(input_list):
    """
    Removes duplicates from a list of strings.

    Args:
    input_list (list): List of strings with possible duplicates.

    Returns:
    list: List with duplicates removed.
    """
    # Use set to keep track of unique strings
    unique_strings = set()

    # List to store the result
    result = []

    for string in input_list:
        # If the string is not already in the set, add it to the result list
        if string not in unique_strings:
            result.append(string)
            unique_strings.add(string)

    return result

In [ ]:
#adding extra country names 
country_to_language.update({
    "Andorra": "Catalan",
    "Montenegro": "Montenegrin",
    "Bosnia": "Bosnian",
    "Scotland": "English",
    "Cyprus": "Greek",
    "Wales": "Welsh",
    "North Macedonia": "Macedonian",
    "N. Macedonia": "Macedonian",
    "Gibraltar": "English",
    "England": "English",
    "San Marino": "Italian",
    "Northern Ireland": "English",
    "North. Ireland": "English",
    "Faroe Islands": "Faroese",
    "Liechtenstein": "German",
    "Czech": "Czech",
    "Korea Republic": "Korean",
    "Ivory Coast": "French"
})

In [ ]:
cyrillic_names_dict = {}

In [ ]:
#cyrillic try 2 
cyrillic_df = pd.DataFrame()

for i in range(2800, 2844): 
    print(f'***{i}***')
    player = []
    part_3_match = ''
    changed_name = ''
    name_found_in_dict = False
    correct_matches = []
    match_case = 'None'
    final_match_status = 'Fail'
    main_word = ''
    status = 'Fail'
    index = i#15
    row = cyrillic_guys.iloc[index]
    this_country_code = row['Team Country Code']
    jersey = row['ORIGINAL JERSEY']
    season = row['Season']
    jersey = ''.join(filter(str.isalpha, jersey))
    country_name = countries_codes[countries_codes[' Code'] == this_country_code]['Country'].unique()[0]

    dataset_nationality = leagues_value[leagues_value['Team 1 Code'] == this_country_code]['Name'].unique()
    dataset_nationality_unidecoded = {unidecode(name) for name in dataset_nationality}

    if i == 530:
        jersey = 'Кашаро Л'
    elif i == 1496:
        jersey = 'John Mintoff'
    elif i == 2416:
        jersey = 'Jackson Porozo'
    elif i == 2427:
        jersey = 'Jackson Porozo'
    elif i == 2476:
        jersey = 'Jackson Porozo'

    if AssemblyHelpers.is_cyrillic(jersey):
        #print('jersey is cyrillic')

        key = f"{jersey}:{country_name}"

        if key in cyrillic_names_dict.keys():
            name = cyrillic_names_dict[key][1]
            name_found_in_dict = True
            status = 'Success'
            print(f'{key} was in dict. got {name}')
        else:
            name, status = find_name_in_language_translate(jersey, 'English')

            html_result = query_google_1(f"{jersey} {country_name} transfermarkt")
            if name == 'Next >':
                name = ''
                if html_result:
                    #print('result')
                    transfermarkt_links = find_transfermarkt_links(html_result)
                    names_links = []
                    correct_names = []
                    #print("Transfermarkt Links:")
                    for link in transfermarkt_links:
                        if link.startswith('/url?q='):
                            link = link.split('/url?q=')[1]
                        name_from_link = extract_name_from_link(link)
                        names_links.append(name_from_link)
                        #print(name_from_link)
                    if len(names_links) == 1:
                        name = names_links[0]
                        #return name
                    elif len(names_links) >= 2:
                        #multiple names found for this name
                        for name_link in names_links:
                            if name_link in dataset_nationality:
                                #print(name_link)
                                correct_names.append(name_link)
                            elif name_link in dataset_nationality_unidecoded:
                                #print(closest_string(name_link, dataset_nationality))
                                correct_names.append(name_link)
                        correct_names = list(set(correct_names))
                        if len(correct_names) == 1:
                            name = correct_names[0]
                            status = 'Success'
                            #return name
                        elif len(correct_names) >= 2:
                            print('multiple matches here', correct_names)
                            #return correct_names
                        else:
                            print('no matches', jersey)
                            status = 'Fail'
                            #return 0
                    else:
                        0==0#nobody found 
                        #return 0
                else:
                    print('nO HTML')
                    0==0#no html

            if name == 'Tip: Search for English results only. You can specify your search language in Preferences':
                name = jersey
                #break
            if 'Did you mean: translate Russian to English:' in name:
                name = name.split('Did you mean: translate Russian to English:')[1]
            elif 'Russian to English:' in name:
                name = name.split('Russian to English:')[1]
            elif 'to English:' in name:
                name = name.split('to English:')[1]
    
            if ((AssemblyHelpers.is_cyrillic(name)) | (name == '')):
                #status = 'Fail'
                name = translate_cyrillic_to_english(jersey)
                if ((AssemblyHelpers.is_cyrillic(name)) | (name == '')):
                    0==0 #it didnt work
                else:
                    status = 'Success'
                    #print('used cyrillic translator, got', name)
    
            if name[-1].isupper(): # Separate capital letters from the rest of the string
                for i in range(len(name)-1, -1, -1):
                    if name[i].islower():
                        main_word = name[:i+1]
                        initial_letters = name[i+1:]
                        break
                name = initial_letters + ' ' + main_word
    
            if name[0] == ' ':
                name = name.lstrip()


            if ((len(name.split(' ')) == 1) & (any(c.isupper() for c in name[1:-1]))):
               # print(name, 'yoooe')
                name = add_spaces_to_middle_caps(name)
            elif any(c.isupper() for c in name[1:-1]):
                name = add_spaces_to_middle_caps(name)
    else:
        #print('jersey not cyrillic ')
        key = f"{jersey}:{country_name}"

        if key in cyrillic_names_dict.keys():
            name = cyrillic_names_dict[key][1]
            name_found_in_dict = True
            status = 'Success'
            print(f'{key} was in dict. got {name}')
        else:
            name = jersey
            status = 'Success'
    #print('gasp')
    #print('b4 checking for cyrillic name, name is ', name)
    if ((AssemblyHelpers.is_cyrillic(name)) | (name == '- none -')):
        print(name, 'MATCH WAS CYRILLIC')
        #print(status)
        name = process_google_query(jersey)
        
        #if this one yields no results try this one below.

        if name == 0: 
    
            name = process_google_query(f"{jersey} {country_name} transfermarkt")

    
    if name == '':
        status = 'Fail'
    elif name[0] == ' ':
        name = name.lstrip()

    #print(f'b4 checking for status, name is {name} and status is {status}')
    if status == 'Success':

        if name_found_in_dict == True:
            player = cyrillic_names_dict[key][0]
        else:
            #print(f'looking for exact. name is {name}, player is {player}')
            if type(name) == list:
                player = name
            else:
                #look for exact matches
                player = [nationality for nationality in dataset_nationality if name in nationality or name in unidecode(nationality)]

                if player != []:
                    player = vet_tokens_names(name, player) #use tokens to vet names 
                    #print('gasp1')

                else:
                    #print(f'no exact matches. name is {name}, player is {player}')
                    player = closest_string(unidecode(name), dataset_nationality_unidecoded)
                    #print('gasp2')

                    if player != []:
                        #print('hir')
                        #print('there were inexact matches', player)
                        player = vet_tokens_names(name, player)
                        #print('gasp30')
                    
                    else:
                        #print(f'no inexact matches either. name is {name}, player is {player}')
                        if 'ch' in name:
                            changed_name = name.replace('ch', 'ć')
                        if 'kh' in name:
                            changed_name = name.replace('kh', 'j')
                        elif 'sell' in name:
                            changed_name = name.replace('sell', 'zzell')
                        elif (('nyan' in name) & (name.endswith('nyan'))):
                            changed_name = name.replace('nyan', 'nian')
                        elif 'es' in name:
                            changed_name = name.replace('es', 'ez')
                        elif 'iy' in name:
                            changed_name = name.replace('iy', 'ij')
                        elif name.endswith('shai'):
                            changed_name = name.replace('shai', 'shaj')
                        elif 'ck' in name:
                            changed_name = name.replace('ck', 'k')
                        elif 'Qu' in name:
                            changed_name = name.replace('Qu', 'Kv')
                        elif 'ny' in name:
                            changed_name = name.replace('ny', 'nj')
                        elif 'Jak' in name:
                            changed_name = name.replace('Jak', 'Xhak')
                        elif 'li' in name:
                            changed_name = name.replace('li', 'lli')
                        elif 'Ji' in name:
                            changed_name = name.replace('Ji', 'Dji')
                            if 'sity' in name:
                                changed_name = changed_name.replace('sity', 'siti')
                        elif "Ga" in name:
                            changed_name = name.replace('Ga', 'Ha')
                        elif 'yevich' in name:
                            changed_name = name.replace('yevich', 'jević')
                        if changed_name != '':
                            player = closest_string(unidecode(changed_name), dataset_nationality_unidecoded)
                            #print('gasp4', player)
                            if player != []:
                                name = changed_name
                                0==0
                                #player = vet_tokens_names(name, player)
                                #print('gasp5')
                            else:
                                player = ''
    print(f'b4 checking for finals, name is {name}, player is {player} and status is {status}')
    if len(player) == 1:
        player_real = threshold_player_match(player[0], dataset_nationality)[0]
    elif player == []:
        player_real = 'No Matches'
    else:
        player_real = 'Multiple matches'
    if type(name) == list:
        name = [word for word in name if word in dataset_nationality or unidecode(word) in dataset_nationality]
    else:
        part_3_match = threshold_player_match(name, dataset_nationality)[0]
        if 'No matches found even with the lowest threshold.' in part_3_match:
            0==0
        else:
            #print('koolio')
            part_3_match = vet_tokens_names(name, part_3_match)

            if part_3_match == []:
                print('vetting tokens removed everybody from part 3 match')
            else:
                0==0
                ##player = part_3_match
                #player_real = part_3_match

    # print('Part 1 gave:', name)
    # print('Part 2 gave', player, player_real)
    # print('Part 3 gave', part_3_match)

    final_match = ''
    if (('No matches found even with the lowest threshold.' in part_3_match) | (part_3_match == [])):
        print(f'X - THRESHOLD MATCH DID NOT WORK. jersey was {jersey}, name was {name}, player was {player} and best match was {player_real}')
    elif player_real == 'No Matches':
        if len(part_3_match) == 1: #if the threshold match returns 1 guy 
            #find an example - ROW 10 SIKERO 
            print(f'X - PLAYER MATCHING DID NOT WORK. jersey was {jersey}, name was {name} and best match was a single, {part_3_match}')
        elif len(part_3_match) >= 2: #if the threshold match returns 2+ guy 
            print(f'X - PLAYER MATCHING DID NOT WORK. jersey was {jersey}, name was {name} and best match was a MULTI, {part_3_match}')
    else:
        #print(f'derp {name} {player} {part_3_match}')

        if len(player) == 1: #part 2 returns 1 player
            if len(part_3_match) == 1: #part 3 returns 1 player
                if ((player_real == part_3_match) | (player_real == part_3_match[0])):
                    final_match = part_3_match
                else:
                    print('parts 2 and 3 each returned 1 name but they arent the same', player_real, part_3_match)
            elif len(part_3_match) >= 2: #part 3 returns 2+ players
                bad_list = []
                #print('gasp1', part_3_match)
                for name_candidate in part_3_match:
                    
                    if name_candidate == player_real[0] or unidecode(name) == player_real[0]:
                        0==0
                    else:
                        bad_list.append(name_candidate)
                        #part_3_match.remove(name_candidate)
                #print('gasp2', bad_list)
                for bad_name in bad_list:
                    part_3_match.remove(bad_name)
                #print('gasp3', part_3_match)
                if part_3_match == []:
                    print('part 2 returned a single, part 3 returned a list, no overlap.')
                else:
                    final_match = part_3_match
                    
        elif len(player) >= 2: #part 2 returns 2+ players
            if len(part_3_match) == 1: #part 3 returns 1 player
                bad_list = []
                if type(player) != str:
                    for name_candidate in player:
                        #print(name)
                        if name_candidate != part_3_match[0]:
                            #print('pikel')
                            bad_list.append(name_candidate)
                    for name_candidate_bad in bad_list:
                        player.remove(name_candidate_bad)
                else:
                    if player_real == part_3_match[0]:
                        final_match = part_3_match
                
                if player == []:
                    print('part 2 returned a list, part 3 returned a single, no overlap.')
                else:
                    player_real = threshold_player_match(player[0], dataset_nationality)
                    final_match = player_real
            elif len(part_3_match) >= 2:
                #print('both attempts returned lists', player, part_3_match)
                #SEE ROW 2. 

                #MAKE A COMBINED SET ALL THE ONES IN BOTH LISTS. 
                final_match = [string for string in part_3_match if string in player or unidecode(string) in player]
                
                #DO A MATCH DATE FILTER with result list. 
            else:
                0==0
    if player == []:
        if threshold_player_match(name, dataset_nationality)[0] == part_3_match:
            final_match = part_3_match
            #print('ceeeeeeele')

    if player == '':
        if threshold_player_match(name, dataset_nationality)[0] == part_3_match:
            final_match = part_3_match

    if ((player == '') & (part_3_match == [])):
        0==0
    else:
        if type(player) == str:
            player = [player]
        elif type(part_3_match) == str:
            part_3_match = [part_3_match]
            #print(part 3 match was a string)
        final_list_to_check = player + part_3_match
        #final_list_to_check = list(set(final_list_to_check))
        if type(name) == list:
            final_list_to_check = final_list_to_check + name
            final_match = [word for word in final_list_to_check if unidecode(word) in dataset_nationality_unidecoded]
            final_match = list(set(final_match))
        else:
            final_match = [word for word in final_list_to_check if unidecode(name) in unidecode(word)]

    if final_match == []:
        if len(part_3_match) == 1:
            part_3_match = part_3_match[0]
        if name == '' or name == []:
            0==0
        else:
            if ((threshold_player_match(name, dataset_nationality)[0] == part_3_match) | (part_3_match in threshold_player_match(name, dataset_nationality)[0])):
                final_match = part_3_match

    if ((final_match != '') & (final_match != [])):
        print(f"jersey said {jersey}. name said {name}")
        print(f'         ***Success*** Final match is {remove_duplicates(final_match)}')  
        key_cyrillic_dict = f"{jersey}:{country_name}"
        if key_cyrillic_dict in cyrillic_names_dict.keys():
            print(f'{key_cyrillic_dict} already in dictionary')
        else:
            cyrillic_names_dict[key_cyrillic_dict] = remove_duplicates(final_match), name
            print(f"added {remove_duplicates(final_match)} and {name} to dict with key {key_cyrillic_dict}")

        if type(final_match) == str:
            final_match = [final_match]
        new_row = {
            'DF index': index,
            'Match ID': row['Match ID'],
            'Competition': row['Competition'],
            'Date': row['Date'],
            'Match': row['Match'],
            'Season': season,
            'Nationality': country_name,
            'Country Code': this_country_code,
            'ORIGINAL JERSEY': row['ORIGINAL JERSEY'],
            'English Name': name,
            'Match(es) Found': remove_duplicates(final_match),
            'Status': status
        }
        cyrillic_df = pd.concat([cyrillic_df, pd.DataFrame([new_row])], ignore_index=True)
        
    else:
        
        print('*********FAIL*********')
        print('Part 1 gave:', name)
        print('Part 2 gave', player, player_real)
        print('Part 3 gave', part_3_match)

        new_row = {
            'DF index': index,
            'Match ID': row['Match ID'],
            'Competition': row['Competition'],
            'Date': row['Date'],
            'Match': row['Match'],
            'Season': season,
            'Nationality': country_name,
            'Country Code': this_country_code,
            'ORIGINAL JERSEY': row['ORIGINAL JERSEY'],
            'English Name': name,
            'Match(es) Found': 0,
            'Status': 'Fail'
        }
        cyrillic_df = pd.concat([cyrillic_df, pd.DataFrame([new_row])], ignore_index=True)

        # final_list_to_check = player + part_3_match
        # print(f'final list to check is {final_list_to_check}')

    #collect output 
        #the index from the cyrillic_guys dataframe
        #date
        #country_name
        #match ID 
        #match 
        #the ORIGINAL JERSEY 
            
        #FINAL MATCH - WHAT IT RETURNED 


#Bugs
        #354 

#plan

    #current - 300 - 400 

        

In [ ]:
cyrillic_df.tail(50)



In [ ]:
big_dataset_cyrillic = cyrillic_df
big_dataset_cyrillic

In [ ]:
big_dataset_cyrillic.to_csv('cyrllic_found_2800_2844.csv')

In [ ]:
row['Match'], row['Date'] #closest_string('Crnigoj', dataset_nationality)

# Left Multiple people

In [ ]:
match_date_multi_guys

# Left 0 People

In [ ]:
match_date_zero_guys

# Making big dataset to add stuff to as we finish them

In [52]:
combined_df_large['Lookup Return Case'].value_counts().head(25)

Lookup Return Case
working                                                                                                                                        64700
Null Name                                                                                                                                       8164
Large Dataset had 0                                                                                                                             5628
Lookup Required                                                                                                                                 3534
Used Large Dataset                                                                                                                              2862
not done yet                                                                                                                                    2844
Online Lookup Required                                                                 

In [80]:
combined_df_edited = combined_df_large[combined_df_large['Lookup Return Case'] != 'Null Name']
combined_df_edited = combined_df_edited[combined_df_edited['Lookup Return Case'] != 'Large Dataset had 0']
combined_df_edited = combined_df_edited[combined_df_edited['Lookup Return Case'] != 'Lookup Required']
combined_df_edited = combined_df_edited[combined_df_edited['Lookup Return Case'] != 'not done yet']
combined_df_edited = combined_df_edited[combined_df_edited['Lookup Return Case'] != 'Online Lookup Required']
combined_df_edited = combined_df_edited[combined_df_edited['Lookup Return Case'] != 'Large Dataset said 0']
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith("Data Ended before season :")]
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith("Data Ended before season :")]
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith("Not in either dataset for currect season but present in both seasons")]
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith("FILTERING USING MATCH DATE LEFT MULTIPLE RESULTS:")]
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith("FILTERING USING MATCH DATE LEFT 0 RESULTS.")]
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith("Was in  DB before or after season")]
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith("asswipe was in DB in before")]

combined_df_edited = combined_df_edited[~((combined_df_edited['Status'] == 'SUCCESS') & 
                                   (combined_df_edited['Lookup Return Case'] == 'Used Large Dataset') & 
                                   (combined_df_edited['Name(s) Found'].astype(str).str.startswith('[')))]#['Lookup Return Case'].value_counts()
combined_df_edited = combined_df_edited[~((combined_df_edited['Status'] == 'SUCCESS') & 
                                   (combined_df_edited['Lookup Return Case'] == 'working') & 
                                   (combined_df_edited['Name(s) Found'].astype(str).str.startswith('[')))]

combined_df_edited = combined_df_edited[combined_df_edited['Name(s) Found'] != '0']
combined_df_edited = combined_df_edited[~combined_df_edited['Lookup Return Case'].str.startswith('Olympics row')]
combined_df_edited

,Unnamed: 0.1,Unnamed: 0,Match ID,Competition,Match,Date,Nationality,Team Country Code,Season,Status,Name,Market Value,Lookup Still Required?,Lookup Return Case,Impute Required?,Name(s) Found,ORIGINAL JERSEY,Match Case
0,0,0,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Anthony Moris,400000.0,False,working,False,Anthony Moris,A Moris,single
1,1,1,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Laurent Jans,1200000.0,False,working,False,Laurent Jans,L Jans,single
4,4,4,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Kevin Malget,150000.0,False,working,False,Kevin Malget,K Malget,single
5,5,5,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Mario Mutsch,125000.0,False,working,False,Mario Mutsch,M Mutsch,single
6,6,6,3133,"World Cup Qualifiers, Europe, 2018",Luxembourg vs France,25-03-2017,Luxembourg,LU,2017,SUCCESS,Chris Philipps,1250000.0,False,working,False,Chris Philipps,C Philipps,single
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96861,96861,1095,1516,Nations League 2018-2020,Faroe Islands vs Azerbaijan,11.10.18,Azerbaijan,AZ,2018,SUCCESS,Araz Abdullayev,500000.0,False,working,False,Araz Abdullayev,A Abdullayev,single
96862,96862,1096,1516,Nations League 2018-2020,Faroe Islands vs Azerbaijan,11.10.18,Azerbaijan,AZ,2018,SUCCESS,Richard Almeyda,1600000.0,False,working,False,Richard Almeyda,Richard Almeida,single
96863,96863,1097,1516,Nations League 2018-2020,Faroe Islands vs Azerbaijan,11.10.18,Azerbaijan,AZ,2018,SUCCESS,Mahir Emreli,1200000.0,False,working,False,Mahir Emreli,Emreli,single
96864,96864,1098,1516,Nations League 2018-2020,Faroe Islands vs Azerbaijan,11.10.18,Azerbaijan,AZ,2018,SUCCESS,Dimitrij Nazarov,500000.0,False,working,False,Dimitrij Nazarov,D Nazarov,single


In [79]:
combined_df_edited[combined_df_edited['Lookup Return Case'].str.startswith('Olympics row')].to_csv('olympics_edge_cases.csv')

In [68]:
combined_df_edited.to_csv('Combined_DF_WORKING - ADD TO THIS.csv')